In [1]:
import pandas as pd
import numpy as np

In [2]:
str_date = "2022-10-28-13"
P_MAX = 50
B_LOW = 0.009
B_HIGH = 0.019
C = 0.019

In [3]:
def genGroupDict(df):
    
    group_dict = dict()

    for i in range(len(df)):
        session_code = str(df.iloc[i]["session.code"])
        round_number = str(df.iloc[i]["subsession.round_number"])
        group_id = str(df.iloc[i]["group.id_in_subsession"])
        tmp_key = session_code + "_" + round_number + "_" + group_id

        if tmp_key not in group_dict:
            group_dict[tmp_key] = {
                "index" : [],
                "item_value": [],
                "group_size" : None,
                "discount_b": None,
                "tick": None,
            }

        item_value = df.iloc[i]["player.item_value"]
        group_dict[tmp_key]["index"].append(i)
        group_dict[tmp_key]["item_value"].append(item_value)

        session_name = df.iloc[i]["session.config.name"]
        if "02" in session_name:
            group_dict[tmp_key]["group_size"] = 2
        elif "05" in session_name:
            group_dict[tmp_key]["group_size"] = 5

        discount_b = df.iloc[i]["session.config.discount_b"]
        group_dict[tmp_key]["discount_b"] = discount_b

        tick = df.iloc[i]["session.config.price_tick"]
        group_dict[tmp_key]["tick"] = tick

    return group_dict

### Honolulu prediction

In [4]:
# df = pd.read_csv("Honolulu_new_" + str_date + ".csv")
# df_h = df.iloc[:, :-13]
df_h = pd.read_csv("Honolulu_paydiff_norm.csv")

In [5]:
def discount(x, t, b):
    if x >= 0:
        if b * t <= 1:
            return x * (1 - b * t)
        else:
            return 0
    else:
        return x * (1 + b * t)

In [6]:
dict_h = genGroupDict(df_h)
df_h[[
    "predict0.player.optimal_dutch_bid",
    "predict0.player.is_dutch_winner",
    "predict0.player.contest_status",
    "predict0.player.english_dropout_elapsed",
    "predict0.player.is_english_winner",
    "predict0.player.is_final_winner",
    "predict0.player.payoff",
    "predict0.group.dutch_time_elapsed",
    "predict0.group.dutch_final_price",
    "predict0.group.english_time_elapsed",
    "predict0.group.english_final_price",
    "predict0.group.final_price",
    "predict0.group.auctioneer_utility",
]] = np.nan

for key in dict_h:
    item_value = dict_h[key]["item_value"]
    if dict_h[key]["group_size"] == 2:
        predict_bid = [0 for i in range(2)]
        if dict_h[key]["discount_b"] == B_HIGH:
            start_price = 21
        elif dict_h[key]["discount_b"] == B_LOW:
            start_price = 12
    elif dict_h[key]["group_size"] == 5:
        predict_bid = [0 for i in range(5)]
        if dict_h[key]["discount_b"] == B_HIGH:
            start_price = 32
        elif dict_h[key]["discount_b"] == B_LOW:
            start_price = 27
    dutch_final_price = np.minimum(start_price, np.max(predict_bid))
    dutch_time_elapsed = (start_price - dutch_final_price) * dict_h[key]["tick"]
    is_dutch_winner = np.array([i >= dutch_final_price for i in predict_bid]).astype(int)
    contest_status = np.array([i >= dutch_final_price for i in item_value]).astype(int)
    if np.sum(contest_status) == 1:
        english_final_price = dutch_final_price
        is_english_winner = contest_status
        english_dropout_elapsed = [np.nan for i in contest_status]
    elif np.sum(contest_status) > 1:
        max_value = np.max(item_value)
        is_english_winner = np.array([i == max_value for i in item_value]).astype(int)
        if np.sum(is_english_winner) > 1:
            english_final_price = max_value
        elif np.sum(is_english_winner) == 1:
            english_final_price = np.max([(i < max_value) * i for i in item_value])
        tmp = []
        for i in range(len(item_value)):
            if contest_status[i]:
                if is_english_winner[i]:
                    if np.sum(is_english_winner) == 1:
                        tmp.append((english_final_price - dutch_final_price) * dict_h[key]["tick"])
                    elif np.sum(is_english_winner) > 1:
                        tmp.append((item_value[i] - dutch_final_price) * dict_h[key]["tick"])
                else:
                    tmp.append((item_value[i] - dutch_final_price) * dict_h[key]["tick"])
            else:
                tmp.append(np.nan)
        english_dropout_elapsed = np.array(tmp)
    english_time_elapsed = (english_final_price - dutch_final_price) * dict_h[key]["tick"]
    is_final_winner = is_english_winner
    final_price = english_final_price
    
    index = dict_h[key]["index"]
    for i in range(len(index)):
        df_h["predict0.player.optimal_dutch_bid"].iat[index[i]] = predict_bid[i]
        df_h["predict0.player.is_dutch_winner"].iat[index[i]] = is_dutch_winner[i]
        df_h["predict0.player.contest_status"].iat[index[i]] = contest_status[i]
        df_h["predict0.player.english_dropout_elapsed"].iat[index[i]] = english_dropout_elapsed[i]
        df_h["predict0.player.is_english_winner"].iat[index[i]] = is_english_winner[i]
        df_h["predict0.player.is_final_winner"].iat[index[i]] = is_final_winner[i]
        df_h["predict0.player.payoff"].iat[index[i]] = discount(dict_h[key]["item_value"][i] - final_price, dutch_time_elapsed + english_time_elapsed, dict_h[key]["discount_b"]) * is_final_winner[i]
        df_h["predict0.group.dutch_time_elapsed"].iat[index[i]] = dutch_time_elapsed
        df_h["predict0.group.dutch_final_price"].iat[index[i]] = dutch_final_price
        df_h["predict0.group.english_time_elapsed"].iat[index[i]] = english_time_elapsed
        df_h["predict0.group.english_final_price"].iat[index[i]] = english_final_price
        df_h["predict0.group.final_price"].iat[index[i]] = final_price
        df_h["predict0.group.auctioneer_utility"].iat[index[i]] = discount(final_price, dutch_time_elapsed + english_time_elapsed, C)

In [7]:
df_h.to_csv("Honolulu_paydiff_norm_zerocost.csv", header=True, index=False)

### Dutch prediction

In [8]:
# df = pd.read_csv("Dutch_new_" + str_date + ".csv")
# df_d = df.iloc[:, :-6]
df_d = pd.read_csv("Dutch_paydiff_norm.csv")

In [9]:
dict_d = genGroupDict(df_d)
df_d[[
    "predict0.player.bid",
    "predict0.player.is_dutch_winner",
    "predict0.player.payoff",
    "predict0.group.dutch_time_elapsed",
    "predict0.group.dutch_final_price",
    "predict0.group.auctioneer_utility",
]] = np.nan

for key in dict_d:
    if dict_d[key]["group_size"] == 2:
        predict_bid = np.array(dict_d[key]["item_value"]) * 0.5
    elif dict_d[key]["group_size"] == 5:
        predict_bid = np.array(dict_d[key]["item_value"]) * 0.8
    final_price = np.max(predict_bid)
    time_elapsed = (P_MAX - final_price) * dict_d[key]["tick"]
    is_winner = np.array([i == final_price for i in predict_bid]).astype(int)
    
    index = dict_d[key]["index"]
    for i in range(len(index)):
        df_d["predict0.player.bid"].iat[index[i]] = predict_bid[i]
        df_d["predict0.player.is_dutch_winner"].iat[index[i]] = is_winner[i]
        df_d["predict0.player.payoff"].iat[index[i]] = discount(dict_d[key]["item_value"][i] - final_price, time_elapsed, dict_d[key]["discount_b"]) * is_winner[i]
        df_d["predict0.group.dutch_time_elapsed"].iat[index[i]] = time_elapsed
        df_d["predict0.group.dutch_final_price"].iat[index[i]] = final_price
        df_d["predict0.group.auctioneer_utility"].iat[index[i]] = discount(final_price, time_elapsed, C)

In [10]:
df_d.to_csv("Dutch_paydiff_norm_zerocost.csv", header=True, index=False)